# Сравнение явной и неявной схемы для волнового уравнения

Обе схемы имеют свои преимущества и недостатки, и выбор между ними зависит от конкретной задачи.

## Явная схема
### Преимущества:
- **Простота реализации** - не требуется решать систему уравнений
- **Вычислительная эффективность** - меньше операций на каждом временном шаге
- **Распараллеливаемость** - легко распараллеливается для многоядерных систем
- **Точность** - хорошая точность при малых шагах сетки

### Недостатки:
- **Условная устойчивость** - работает только при r ≤ 1 (число Куранта)
- **Ограничение на шаг по времени** - требуется мелкий шаг τ

## Неявная схема
### Преимущества:
- **Безусловная устойчивость** - при σ ≥ 0.5 устойчива при любом шаге по времени
- **Большие шаги по времени** - можно использовать существенно больший шаг τ
- **Регулируемые свойства** - параметр σ позволяет контролировать точность и диссипацию

### Недостатки:
- **Сложность реализации** - требуется решать СЛАУ на каждом шаге
- **Вычислительные затраты** - больше операций на шаг
- **Нелинейность** - сложнее расширить на нелинейные задачи

## Какая схема лучше?

1. **Явная схема лучше, когда**:
   - Требуется высокая скорость вычислений для небольших временных интервалов
   - Длина струны невелика или скорость волны мала
   - Важна простота реализации

2. **Неявная схема лучше, когда**:
   - Необходимо моделировать длительные процессы
   - Скорость волны высока или длина струны велика
   - Критична устойчивость решения
   - При σ = 0.5 (схема Кранка-Николсона) обеспечивает второй порядок точности по времени

## Вывод
Для учебных задач и чистого волнового уравнения **явная схема** часто предпочтительнее из-за своей простоты и эффективности при соблюдении условия устойчивости.

Для реальных приложений, где нужны большие области или большие временные интервалы, **неявная схема** с σ = 0.5 (Кранка-Николсона) часто является оптимальным выбором благодаря сочетанию устойчивости и точности.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Layout, Dropdown, Output
from IPython.display import display, clear_output

In [9]:
def get_task_conditions(task, L, c):
    """
    Возвращает функции начальных и граничных условий для выбранной задачи.
    """
    if task == 1:
        # Задача №1: Колебания струны с начальным отклонением
        u0_func = lambda x: np.sin(x)
        v0_func = lambda x: np.zeros_like(x)
        f_func = lambda x, t: 0
        mu0_func = lambda t: 0
        muL_func = lambda t: 0
        analytical_solution = lambda x, t: np.sin(x) * np.cos(c * t)
    elif task == 2:
        # Задача №2: Колебания струны с начальным отклонением в виде cos(mx)
        m = 3  # Параметр m
        u0_func = lambda x: np.cos(m * x)
        v0_func = lambda x: c * m * np.sin(m*x)
        f_func = lambda x, t: 0
        mu0_func = lambda t: np.cos(c * m * t)
        muL_func = lambda t: np.cos(c * m * t)
        analytical_solution = lambda x, t: np.cos(m * (x - c * t))
    elif task == 3:
        # Задача №3: Колебания струны с начальным "треугольным" отклонением
        x1, x2 = L / 3, 2 * L / 3  # Параметры x1 и x2
        
        def triangle_function(x):
            result = np.zeros_like(x)
            for i, xi in enumerate(x):
                if 0 <= xi <= x1:
                    result[i] = 0
                elif x1 < xi <= (x1 + x2) / 2:
                    result[i] = 2 * (xi - x1) / (x2 - x1)
                elif (x1 + x2) / 2 < xi <= x2:
                    result[i] = 2 * (xi - x2) / (x2 - x1)
                elif x2 < xi <= L:
                    result[i] = 0
            return result
            
        u0_func = triangle_function
        v0_func = lambda x: np.zeros_like(x)
        f_func = lambda x, t: 0
        mu0_func = lambda t: 0
        muL_func = lambda t: 0
        analytical_solution = lambda x, t: (triangle_function(x + c*t) + triangle_function(x - c*t)) / 2
    elif task == 4:
        # Задача №4: Колебания струны с источником
        def wave_function(x, t, c):
            u = np.zeros_like(x)
            for i, xi in enumerate(x):
                if t <= xi / c:
                    u[i] = 0
                else:
                    u[i] = np.sin(c * (t - xi / c))
            return u
    
        u0_func = lambda x: np.zeros_like(x)
        v0_func = lambda x: np.zeros_like(x)
        f_func = lambda x, t: 0
        mu0_func = lambda t: np.sin(c * t)
        muL_func = lambda t: 0
        analytical_solution = lambda x, t: wave_function(x, t, c)
    return u0_func, v0_func, f_func, mu0_func, muL_func, analytical_solution

In [14]:
def solve_wave_equation_explicit(L, T, c, Nx, Nt, u0_func, v0_func, f_func, mu0_func, muL_func, sigma=0.5):
    """
    Решение уравнения колебаний струны явной разностной схемой.
    
    Параметры:
        L (float): Длина струны
        T (float): Время моделирования
        c (float): Скорость распространения волны
        Nx (int): Число узлов по пространству
        Nt (int): Число узлов по времени
        u0_func (function): Начальное условие u(x, 0)
        v0_func (function): Начальное условие u_t(x, 0)
        f_func (function): Источник f(x, t)
        mu0_func (function): Граничное условие u(0, t)
        muL_func (function): Граничное условие u(L, t)
        sigma (float): Параметр схемы (0 <= sigma <= 1)
    """
    # Шаги по пространству и времени
    h = L / (Nx - 1)
    tau = T / (Nt - 1)
    r = (c * tau / h) ** 2  # Число Куранта в квадрате
    
    # Проверка устойчивости
    if r > 1:
        print(f"Предупреждение: r={r:.4f} > 1, явная схема может быть неустойчива.")
    else:
        print(f"r={r:.4f} ≤ 1, явная схема устойчива.")
    
    # Сетка
    x = np.linspace(0, L, Nx)
    t = np.linspace(0, T, Nt)
    
    # Инициализация массивов
    u = np.zeros((Nt, Nx))
    u0 = u0_func(x)
    v0 = v0_func(x)
    u[0, :] = u0
    
    # Первый временной слой
    u[1, 1:-1] = (
        u0[1:-1]
        + tau * v0[1:-1]
        + (r / 2) * (u0[2:] - 2 * u0[1:-1] + u0[:-2])
        + (tau**2 / 2) * f_func(x[1:-1], t[0])
    )
    u[1, 0] = mu0_func(t[1])
    u[1, -1] = muL_func(t[1])
    
    # Применение явной схемы для следующих временных слоев
    for n in range(1, Nt - 1):
        for i in range(1, Nx - 1):
            u[n + 1, i] = (
                2 * (1 - r) * u[n, i]
                - u[n - 1, i]
                + r * (u[n, i + 1] + u[n, i - 1])
                + tau**2 * f_func(x[i], t[n])
            )
        # Граничные условия
        u[n + 1, 0] = mu0_func(t[n + 1])
        u[n + 1, -1] = muL_func(t[n + 1])
    
    return x, t, u, r**0.5, sigma

In [15]:
def solve_wave_equation_implicit(L, T, c, Nx, Nt, u0_func, v0_func, f_func, mu0_func, muL_func, sigma=0.5):
    """
    Решение уравнения колебаний струны неявной схемой с прогонкой.
    
    Параметры аналогичны явной схеме, но sigma контролирует "вес" неявной части (0 <= sigma <= 1)
    sigma = 0.5 даёт схему Кранка-Николсона (второй порядок по времени)
    sigma = 1 даёт полностью неявную схему
    """
    # Шаги по пространству и времени
    h = L / (Nx - 1)
    tau = T / (Nt - 1)
    r = (c * tau / h) ** 2  # Число Куранта в квадрате
    
    # Сетка
    x = np.linspace(0, L, Nx)
    t = np.linspace(0, T, Nt)
    
    # Инициализация массивов
    u = np.zeros((Nt, Nx))
    u0 = u0_func(x)
    v0 = v0_func(x)
    u[0, :] = u0
    
    # Первый временной слой (используем явную схему)
    u[1, 1:-1] = (
        u0[1:-1]
        + tau * v0[1:-1]
        + (r / 2) * (u0[2:] - 2 * u0[1:-1] + u0[:-2])
        + (tau**2 / 2) * f_func(x[1:-1], t[0])
    )
    u[1, 0] = mu0_func(t[1])
    u[1, -1] = muL_func(t[1])
    
    # Применение неявной схемы для следующих временных слоев
    for n in range(1, Nt - 1):
        # Коэффициенты для трехдиагональной матрицы
        a = np.ones(Nx) * (-sigma * r)  # Коэффициенты при u_{n+1,i-1}
        b = np.ones(Nx) * (1 + 2 * sigma * r)  # Коэффициенты при u_{n+1,i}
        c = np.ones(Nx) * (-sigma * r)  # Коэффициенты при u_{n+1,i+1}
        
        # Правая часть системы уравнений
        d = np.zeros(Nx)
        for i in range(1, Nx - 1):
            d[i] = (
                (1 - sigma) * r * u[n, i-1] + 
                (2 - 2 * (1 - sigma) * r) * u[n, i] + 
                (1 - sigma) * r * u[n, i+1] - 
                u[n-1, i] + 
                tau**2 * f_func(x[i], t[n])
            )
        
        # Граничные условия
        d[0] = mu0_func(t[n+1])
        d[-1] = muL_func(t[n+1])
        
        # Прогонка для решения системы уравнений
        # Прямой ход
        alpha = np.zeros(Nx)
        beta = np.zeros(Nx)
        
        alpha[1] = -c[0] / b[0]
        beta[1] = d[0] / b[0]
        
        for i in range(1, Nx-1):
            denom = b[i] + a[i] * alpha[i]
            alpha[i+1] = -c[i] / denom
            beta[i+1] = (d[i] - a[i] * beta[i]) / denom
        
        # Обратный ход
        u[n+1, -1] = d[-1]  # Правая граница
        for i in range(Nx-2, -1, -1):
            u[n+1, i] = alpha[i+1] * u[n+1, i+1] + beta[i+1]
    
    return x, t, u, r**0.5, sigma

In [42]:
def calculate_convergence_order(task, L=2 * np.pi, T=2, c=1, sigma=0.5):
    """
    Расчет порядка сходимости численной схемы.
    
    Возвращает:
        - список шагов сетки
        - список ошибок
        - оценки порядков точности
    """
    u0_func, v0_func, f_func, mu0_func, muL_func, analytical_solution = get_task_conditions(task, L, c)
    
    # Различные размеры сеток для анализа
    Nx_values = [10, 20, 40, 80, 160]
    h_values = [L / (nx - 1) for nx in Nx_values]
    errors = []
    
    for Nx in Nx_values:
        # Поддерживаем постоянное число Куранта (отношение tau/h)
        Nt = int(Nx * T / L) + 1
        
        x, t, u, _, _ = solve_wave_equation_implicit(L, T, c, Nx, Nt, u0_func, v0_func, f_func, mu0_func, muL_func, sigma)
        
        # Сравнение с аналитическим решением в конечный момент времени
        numerical = u[-1, :]
        analytical = analytical_solution(x, T)
        error = np.linalg.norm(numerical - analytical) / np.linalg.norm(analytical)
        errors.append(error)
    
    # Расчет порядков точности
    orders = []
    for i in range(1, len(errors)):
        p = np.log(errors[i-1] / errors[i]) / np.log(Nx_values[i] / Nx_values[i-1])
        orders.append(p)
    
    # Вывод и визуализация результатов
    plt.figure(figsize=(10, 6))
    plt.loglog(h_values, errors, 'bo-', linewidth=2, label='Ошибка')
    
    # Линии идеальных порядков точности
    h_ref = np.array([min(h_values), max(h_values)])
    plt.loglog(h_ref, [errors[0] * (h_ref[0]/h_values[0])**1, errors[0] * (h_ref[1]/h_values[0])**1], 'k--', label='O(h)')
    plt.loglog(h_ref, [errors[0] * (h_ref[0]/h_values[0])**2, errors[0] * (h_ref[1]/h_values[0])**2], 'g--', label='O(h²)')
    
    plt.xlabel('Шаг сетки (h)', fontsize=12)
    plt.ylabel('Относительная ошибка', fontsize=12)
    plt.title(f'Анализ сходимости для задачи №{task}', fontsize=14)
    plt.grid(True, which='both', linestyle='--')
    plt.legend()
    
    # Вывод численных значений
    print(f"{'Шаг h':10s} | {'Ошибка':15s} | {'Порядок точности':20s}")
    print("-" * 50)
    for i, (h, err) in enumerate(zip(h_values, errors)):
        order_str = f"{orders[i-1]:.2f}" if i > 0 else "—"
        print(f"{h:.6f}  | {err:.6e} | {order_str:20s}")
    
    plt.show()
    return h_values, errors, orders

In [43]:
def plot_results(task, L=2 * np.pi, T=2, c=1, Nx=100, Nt=200, time_index=50, sigma=1, calc_order=True):
    """
    Визуализация результатов расчета с добавлением оценки порядка точности.
    
    Параметры:
        task: номер задачи
        L: длина струны
        T: конечное время
        c: скорость распространения волны
        Nx, Nt: число узлов по пространству и времени
        time_index: индекс времени для отображения
        sigma: параметр схемы
        calc_order: рассчитывать ли порядок точности
    """
    u0_func, v0_func, f_func, mu0_func, muL_func, analytical_solution = get_task_conditions(task, L, c)
    x, t, u, r, sigma = solve_wave_equation_implicit(L, T, c, Nx, Nt, u0_func, v0_func, f_func, mu0_func, muL_func, sigma)

    # Вычисление ошибки
    time = t[time_index]
    numerical = u[time_index, :]
    analytical = analytical_solution(x, time)
    error = np.linalg.norm(numerical - analytical) / np.linalg.norm(analytical)
    
    # Расчет порядка точности, если запрошено
    order_text = ""
    if calc_order:
        # Вычисляем решение на более мелкой сетке
        Nx2 = 2 * Nx
        Nt2 = 2 * Nt
        x2, t2, u2, _, _ = solve_wave_equation_implicit(L, T, c, Nx2, Nt2, u0_func, v0_func, f_func, mu0_func, muL_func, sigma)
        
        # Сопоставляем время
        time_index2 = min(time_index * 2, len(t2) - 1)
        time2 = t2[time_index2]
        
        # Вычисляем ошибку на новой сетке
        numerical2 = u2[time_index2, :]
        analytical2 = analytical_solution(x2, time2)
        error2 = np.linalg.norm(numerical2 - analytical2) / np.linalg.norm(analytical2)
        
        # Оценка порядка точности по пространству
        if error2 > 0 and error > 0:  # Избегаем деления на ноль или логарифма от отрицательного числа
            p_x = np.log(error / error2) / np.log(2)
            order_text = f"Порядок точности по пространству: {p_x:.2f}"
    
    # Построение графиков
    fig, ax1 = plt.subplots(1, 1, figsize=(18, 6))

    ax1.plot(x, numerical, 'b-', linewidth=2, label=f"Численное решение")
    ax1.plot(x, analytical, 'r--', linewidth=2, label=f"Аналитическое решение")
    ax1.set_xlabel("x", fontsize=12)
    ax1.set_ylabel("u(x, t)", fontsize=12)
    title = f"Решения для задачи №{task} (t={time:.2f}) c r={r:.2f}, sigma={sigma:.2f}\n"
    title += f"Относительная ошибка: {error:.2e}"
    if order_text:
        title += f"\n{order_text}"
    ax1.set_title(title, fontsize=14)
    ax1.legend(fontsize=12)
    ax1.grid(True)

    plt.tight_layout()
    plt.show()

In [44]:
output_widget = Output()

task_slider = IntSlider(min=1, max=4, step=1, value=1, description='Задача №:', style={'description_width': '100px'})
L_slider = FloatSlider(min=np.pi, max=4*np.pi, step=np.pi, value=2*np.pi, description='Длина струны L:', style={'description_width': '100px'})
T_slider = FloatSlider(min=0.5, max=5, step=0.5, value=2, description='Время T:', style={'description_width': '100px'})
c_slider = FloatSlider(min=0.1, max=2, step=0.1, value=1, description='Скорость c:', style={'description_width': '100px'})
Nx_slider = IntSlider(min=10, max=1000, step=10, value=100, description='Узлы по x:', style={'description_width': '100px'})
Nt_slider = IntSlider(min=10, max=1000, step=10, value=100, description='Узлы по t:', style={'description_width': '100px'})
time_slider = IntSlider(min=0, max=999, step=1, value=50, description='Время (индекс):', style={'description_width': '100px'})
sigma_slider = FloatSlider(min=0, max=1.5, step=0.1, value=0.5, description='Параметр sigma:', style={'description_width': '100px'})

interact(
    plot_results,
    task=task_slider,
    L=L_slider,
    T=T_slider,
    c=c_slider,
    Nx=Nx_slider,
    Nt=Nt_slider,
    time_index=time_slider,
    sigma=sigma_slider
);

display(output_widget)

interactive(children=(IntSlider(value=1, description='Задача №:', max=4, min=1, style=SliderStyle(description_…

Output()

In [45]:
interact(
    calculate_convergence_order,
    task=task_slider,
    L=L_slider,
    T=T_slider,
    c=c_slider,
    Nx=Nx_slider,
    Nt=Nt_slider,
    time_index=time_slider,
    sigma=sigma_slider
);

display(output_widget)

interactive(children=(IntSlider(value=1, description='Задача №:', max=4, min=1, style=SliderStyle(description_…

Output()

# Почему для треугольных начальных условий отличается порядок точности

Для "треугольных" начальных условий (задача №3) порядок точности численного решения действительно ниже, чем для гладких функций (например, синусов или косинусов). Это объясняется несколькими важными причинами:

## Причины снижения порядка точности

### 1. Недостаточная гладкость решения

Треугольная функция содержит точки излома, где первая производная терпит разрыв:
- В вершине треугольника
- В точках соединения треугольника с нулевым участком

В этих точках вторая производная не существует или имеет сингулярность.

### 2. Теоретическое обоснование

Теоретически доказано, что для центральной разностной схемы:
- Если решение имеет непрерывные производные до 4-го порядка — достигается 2-й порядок точности
- Если решение имеет разрывы в производных — порядок точности снижается

### 3. Распространение особенностей

При решении волнового уравнения точки излома начальных условий распространяются вдоль характеристик:
- Образуются линии пониженной гладкости в пространственно-временной области
- Эти линии соответствуют движению особенностей со скоростью ±c

### 4. Измеряемый порядок сходимости

Для треугольных начальных условий часто наблюдается:
- Порядок точности между 1 и 2
- При более грубых сетках — ближе к 1
- При более мелких сетках — может приближаться к 1.5

## Практические выводы

1. **Адаптивные сетки**: Для повышения точности решения с треугольными начальными условиями эффективно использовать адаптивные сетки с измельчением вблизи особенностей.

2. **Схемы высокого порядка**: Схемы высокого порядка (например, WENO) лучше справляются с моделированием разрывных решений.

3. **Влияние на практические задачи**: В реальных физических задачах треугольные формы часто аппроксимируют гладкими функциями для улучшения сходимости численных методов.

Это классический пример в численных методах, демонстрирующий, что порядок точности зависит не только от схемы, но и от гладкости решения.